In [1]:
using PyCall
using NPZ, JLD2

py"""
import pickle
import numpy as np

def load_pickle(fpath):
    with open(fpath, "rb") as f:
        data = pickle.load(f)
    return data
"""

load_pickle = py"load_pickle"

PyObject <function load_pickle at 0x7f1b567911f0>

In [2]:
cosmo_params = load_pickle("cosmo_params.pkl") # cosmo_params is a dictionary

box = "Box0_1400"
h = cosmo_params[box]["H0"]/100

0.59729599

In [3]:
print("/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/$(box)_Pk.pkl")

/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/Box0_1400_Pk.pkl

In [5]:
Pk_fname = "/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/$(box)_Pk.pkl"
Pkz = load_pickle(Pk_fname) # Pkz is a dictionary of functions

dndM_fname = "/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/$(box)_dndM.pkl"
dndMs = load_pickle(dndM_fname) # dndMs is a dictionary of dictionaries

NvM_fname = "/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/$(box)_NvsM.pkl"
NvMs = load_pickle(NvM_fname) # NvMs is a dictionary of dictionaries

Dict{Any, Any} with 16 entries:
  0.953321 => Dict{Any, Any}("vol"=>1.15762e9, "Mpart"=>4.93984e10, "M"=>[5.511…
  0.620002 => Dict{Any, Any}("vol"=>1.15762e9, "Mpart"=>4.93984e10, "M"=>[5.431…
  0.715608 => Dict{Any, Any}("vol"=>1.15762e9, "Mpart"=>4.93984e10, "M"=>[5.460…
  0.65036  => Dict{Any, Any}("vol"=>1.15762e9, "Mpart"=>4.93984e10, "M"=>[5.444…
  0.403224 => Dict{Any, Any}("vol"=>1.15762e9, "Mpart"=>4.93984e10, "M"=>[5.285…
  0.537169 => Dict{Any, Any}("vol"=>1.15762e9, "Mpart"=>4.93984e10, "M"=>[5.382…
  0.25     => Dict{Any, Any}("vol"=>1.15762e9, "Mpart"=>4.93984e10, "M"=>[5.160…
  1.0      => Dict{Any, Any}("vol"=>1.15762e9, "Mpart"=>4.93984e10, "M"=>[5.513…
  0.366459 => Dict{Any, Any}("vol"=>1.15762e9, "Mpart"=>4.93984e10, "M"=>[5.258…
  0.787402 => Dict{Any, Any}("vol"=>1.15762e9, "Mpart"=>4.93984e10, "M"=>[5.480…
  0.563471 => Dict{Any, Any}("vol"=>1.15762e9, "Mpart"=>4.93984e10, "M"=>[5.395…
  0.302679 => Dict{Any, Any}("vol"=>1.15762e9, "Mpart"=>4.93984e10, "M"=>[5.1

In [29]:
z_to_a = Dict(zip(sort(collect(keys(Pkz)), rev=true), sort(collect(keys(dndMs)))))
a_to_z = Dict(zip(sort(collect(keys(dndMs))), sort(collect(keys(Pkz)), rev=true)))

Dict{Any, Any} with 16 entries:
  0.953321 => 0.0489646
  0.620002 => 0.612898
  0.715608 => 0.397413
  0.302679 => 2.30383
  0.65036  => 0.53761
  0.403224 => 1.48001
  0.866399 => 0.154203
  0.537169 => 0.861612
  0.25     => 3.0
  1.0      => 0.0
  0.512095 => 0.952763
  0.591061 => 0.691873
  0.366459 => 1.72882
  0.787402 => 0.269999
  0.563471 => 0.774714
  0.488191 => 1.04838

In [114]:
import Pkg; Pkg.add("QuadGK")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed QuadGK ───────── v2.8.2
   Installed DataStructures ─ v0.18.13
    Updating `~/.julia/environments/v1.8/Project.toml`
  [1fd47b50] + QuadGK v2.8.2
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [864edb3b] + DataStructures v0.18.13
  [1fd47b50] + QuadGK v2.8.2
Precompiling project...
  ✓ DataStructures
  ✓ QuadGK
  2 dependencies successfully precompiled in 5 seconds. 40 already precompiled.


In [122]:
using DoubleExponentialFormulas
using QuadGK
#stealing from Komatsu 
function sigma2(pk, R::Real)
   function dσ2dk(k)
      x = k * R
      W = (3 / x) * (sin(x) / x^2 - cos(x) / x)
      dσ2dk = W^2 * pk(k) * k^2 / 2 / π^2
   end
   res, err = quadde(dσ2dk, 0, 20 / R)
   return res
end

function dsigma2dR(pk, R::Real)
   function dσ2dRdk(k)
      x = k * R
      W = (3 / x) * (sin(x) / x^2 - cos(x) / x)
      dWdx = (-3 / x) * ((3 / x^2 - 1) * sin(x) / x - 3 * cos(x) / x^2)
      dσ2dRdk = 2 * W * dWdx * pk(k) * k^3 / 2 / π^2
   end
   res, err = quadde(dσ2dRdk, 0, 20 / R)
   return res
end

dsigma2dR (generic function with 1 method)

In [123]:

const G = 4.3009e-9 #km^2 Mpc/ (Msolar  s^2) weird units to make rhom_a good units 
cosmo_params = load_pickle("cosmo_params.pkl")

function M_to_R(M, box, a)
    return (M / (4/3 * π * rhom_a(box, a))) ^ (1/3) # h^-1 Mpc  
end

function R_to_M(R,box, a)
    return R ^ 3 * 4/3 * π * rhom_a(box, a)
end

function dRdM(M, box, a)
    return 1/(6^(2/3)* π^(1/3) * M^(2/3) * rhom_a(box, a)^(1/3))
end

function scaleToRedshift(a)
    return 1/a-1
end

function redshiftToScale(z)
    return 1/(1+z)
end


function logspace(start, stop, n)
        oup = (range(log10(start),stop=stop,length=n))
        return [10^i for i in oup]
end

function rhom_a(box, a)
    ombh2 = cosmo_params[box]["ombh2"]
    omch2 = cosmo_params[box]["omch2"]
    H0 = cosmo_params[box]["H0"] #[km s^-1 Mpc-1]
    h = H0/100
    
    Ωm = ombh2/h^2 + omch2/h^2
    ΩΛ = 1 - Ωm
    ρcrit0 = 3*H0^2/(8*π*G) # h^2 Msol/Mpc^3
    
    return Ωm*ρcrit0*(Ωm*a^(-3) + ΩΛ) * a^(3) #times a^3 bc rho in comoving(?)
end

rhom_a (generic function with 1 method)

In [124]:
data = []
N_data = []
using Interpolations

vol = -1
Mpart = -1

dlnσinvdMs = Dict()
for z in zip(keys(z_to_a))
    z = (collect(z)[1])
    a = z_to_a[z]
    Pk = Pkz[z]
    c_data = NvMs[a]

    Ms = c_data["M"]
    N = c_data["N"]
    Ngt10 = findall(N .> 10)
    edges = c_data["bin_edges"]

    if vol == -1
        vol = c_data["vol"]
    end
    @assert vol == c_data["vol"]

    if Mpart == -1
        Mpart = c_data["Mpart"]
    end
    @assert Mpart == c_data["Mpart"]

    edge_pairs = [[edges[i], edges[i+1]] for i in 1:length(edges)-1]
    edge_pairs = edge_pairs[Ngt10, :]

    @assert length(edge_pairs) == length(Ms)
    @assert length(N) == length(Ms)

    datapoint = Dict()
    
    for (N_cur, M_cur, edge_pair) in zip(N, Ms, eachrow(edge_pairs))
        datapoint["a"] = a
        datapoint["M"] = M_cur
        datapoint["edges"]  = edge_pair
        push!(N_data, N_cur)
        push!(data, copy(datapoint))
    end

    
    sM = logspace(log10(100*Mpart), 17, 50)
    R  = [M_to_R(m, box, a) for m in sM]
    println("a")
    sigma2s = collect(Iterators.flatten(sigma2.(Pk,R)))
    println("b") 
    dsigma2dRs = collect(Iterators.flatten(dsigma2dR.(Pk,R)))
    dRdMs = collect(Iterators.flatten(dRdM.(sM, box, a)))
    println("c") 
    dlnσinvdM = -1/2* (dsigma2dRs./sigma2s).*dRdMs
    println("d") 



    f_dlnsinvdM_log   = LinearInterpolation(log10.(sM), dlnsinvdM, extrapolation_bc=Periodic())
#     f_dlnsinvdM = x -> f_dlnsinvdM_log(log10(x))

#     dlnσinvdMs[a] = f_dlnsinvdM

#     f_M = logspace(log10(minimum(Ms)), log10(maximum(Ms)-1),100)

#         plt.plot(sM, f_dlnsinvdM.(sM))
#         plt.plot(sM, dlnsinvdM, "x")
#         plt.xscale("log")
#         plt.yscale("log")

#     plt.show()
end

a
b
c
d
a
b


LoadError: InterruptException:

In [125]:
using Pkg
Pkg.add("Interpolations")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [121]:
vec1 = [1,2,3]
vec2 = [4,5,6]
vec1.*vec2

3-element Vector{Int64}:
  4
 10
 18